# Analysis of the State of the Business

## Credit Product Diagram

<img src="https://i.imgur.com/B7NvEKb.png">

> **Note**: Past debt delinquency rates (DQ) have determined the interest rates for the Contigo product. 
>
> The monthly DQ rates for each Contigo loan type is an estimation that holds throught this year.

In [5]:
# Contigo A loan features
CA_features = {
    "loan_amount_mxn":1000,
    "payment_term_in_months":1,
    "monthly_interest_rate":0.15,
    "monthly_dq_rate":0.135
}

# Contigo B loan features
CB_features = {
    "loan_amount_mxn":2000,
    "payment_term_in_months":1,
    "monthly_interest_rate":0.1,
    "monthly_dq_rate":0.08
}

In [6]:
# Amount of customers/accounts per loan type on the first month
CA_accounts = 5000

CB_accounts = 10000

In [7]:
# Estimated growth rates per loan type
CA_growth_rate_mom = 0.25

CB_growth_rate_mom = 0.1